In [1]:
import os, re, json , glob

In [2]:
# import re
# from bs4 import BeautifulSoup

# # Sample README content (you can load this from a file instead)
# # readme_content = '''
# # <details>
# # <summary>Click the arrow to view options.</summary>

# #   - m: metre (unit for length).
# #   - Pa: pascal (unit for pressure).
# #   - K: kelvin (unit for temperature).

# # </details>
# # '''

def extract_bullets_with_brackets(html_text):
    soup = BeautifulSoup(html_text, "html.parser")
    results = {}

    bullet_pattern = re.compile(r"\s*-\s*(\w+):\s*([^\(]+?)(?:\s*\((.*?)\))?\.")

    for details in soup.find_all("details"):
        lines = details.get_text().splitlines()
        for line in lines:
            match = bullet_pattern.match(line)
            if match:
                symbol, description, bracket_info = match.groups()
                results[symbol] = {
                    "text1": description.strip(),
                    "text2": bracket_info.strip() if bracket_info else None
                }

    return results

# # # Run the function
# # bullets = extract_bullets_with_brackets(readme_content)

# # # Print neatly
# # from pprint import pprint
# # pprint(bullets)


In [3]:
rme = glob.glob('*/*README*')
rme

['native-horizontal-grid-type/README.md',
 'source-type/README.md',
 'native-horizontal-grid-region/README.md',
 'model-calendar/README.md',
 'license/README.md',
 'frequency/README.md',
 'activity/README.md',
 'native-horizontal-grid-temporal-refinement/README.md',
 'product/README.md',
 'mip/README.md',
 'grid-label/README.md',
 'model-component-type/README.md',
 'resolution/README.md',
 'native-vertical-grid-coordinate/README.md',
 'organisation/README.md',
 'realm/README.md',
 'native-vertical-grid-units/README.md',
 'model-family/README.md']

In [4]:
# pm = list(DATA_DESCRIPTOR_CLASS_MAPPING.values())[-4]


def bullet_pydantic(pm):

    keys = ""

    for key, value in pm.__pydantic_fields__.items():
        rq = '✓' if value.is_required() else '✗'
        
        typename = getattr(value.annotation, '__name__', str(value.annotation))
        description = value.description or '<< No description in pydantic model (see esgvoc) >>'

        # print(f"- **`{key}`**  [**{typename}**]  {rq} : \n  {description}\n")
        # print(f"- `{key}` [{typename}] {rq} : {description}")
        # print(f"- `{key}`\n      Type       : [{typename}]\n      Required   : {rq}\n      Description: {description}\n")
        
        # print(f"- **`{key}`**  \n  {rq} (**{typename}**)\n  {description.rstrip()}\n  ")
        
        keys += f"- **`{key}`** (**{typename}**) \n  {description.rstrip()}\n"
        
        # keys += f"- **`{key}`**  \n  {rq} (**{typename}**)\n  {description.rstrip()}\n  "
        
        
    return keys

def bullet_names(keynames):
    keys = ""
    for key in keynames:
        # print(f"- `{key}`")
        print(f"- **`{key}`**")
        keys += f"- **`{key}`**  \n  ? (**NoType**)\n  No Linked Pydantic Model \n"
    return keys


def extract_description(readme_content):
    """Extract description from README file content."""
    # Pattern to match the description section
    pattern = r'<section id="description">(.*?)</section>'
    match = re.search(pattern, readme_content, re.DOTALL)
    
    if not match:
        return None
    
    section_content = match.group(1).strip()
    
    # Extract just the description text after "## Description"
    desc_pattern = r'## Description\s*(.*?)(?=\n\n|\Z)'
    desc_match = re.search(desc_pattern, section_content, re.DOTALL)
    
    return desc_match.group(1).strip() if desc_match else None

In [5]:
# for rd in rme:
#     path = os.path.dirname(rd.replace('README.md',''))
#     print(path)
#     mytype = path.split('/')[-1]
    
#     context = {
#     "@context": [
#         "../_context_",
#         {
#             "@base": f"https://wcrp-cmip.github.io/WCRP-universe/{mytype}/",
#             "@vocab": f"https://wcrp-cmip.github.io/WCRP-universe/{mytype}/"
#         }
#     ],
#     "@embed": "@always",
#     "@type": [
#         "universal",
#         f"wcrp:{mytype}",
#     ]
#     }
#     with open(f'./{path}/_context_', 'w') as f:
#         json.dump(context, f, indent=4)
    
#     branches = extract_bullets_with_brackets(open(rd).read())
#     # print(branches)
    
#     for k, v in branches.items():
#         validation = k.replace('_','-').replace(' ','-')
#         filename = validation.lower()
        
        
#         ld = {
#             "id": filename,
#             "type": [f"wcrp:{mytype}","universal"],
#             "validation-key": validation,

            
#         } 
        
#         if v['text2'] is not None:
            
#             ld['description'] =  v['text2']
#             ld['ui-label'] = v['text1']
            
#         else:
#             ld['description'] = v['text1']
#             ld['ui-label'] = validation.replace('-',' ').capitalize()
            
#         # print(ld)
        
#         # with open(f'./{path}/{filename}.json', 'w') as f:
#         #     json.dump(ld, f, indent=4)

In [6]:
folders = glob.glob('*/')
folders

['native-horizontal-grid-type/',
 'source-type/',
 'native-horizontal-grid-region/',
 'empty/',
 'model-calendar/',
 'license/',
 'frequency/',
 'activity/',
 'native-horizontal-grid-temporal-refinement/',
 'product/',
 'mip/',
 'grid-label/',
 'model-component-type/',
 'resolution/',
 'native-vertical-grid-coordinate/',
 'organisation/',
 'realm/',
 'native-vertical-grid-units/',
 'model-family/']

In [20]:
import numpy as np
import esgvoc
from esgvoc.api.data_descriptors import DATA_DESCRIPTOR_CLASS_MAPPING

from cmipld.utils.git import get_path_url,get_repo_url,get_relative_path,url2io
from cmipld import prefix_url,resolve_prefix

missing_pydantic = []

for dir in folders:
    
    print(dir)
    name = dir.strip('/')
    
    json_files = [f for f in os.listdir(dir) if f.endswith('.json')]
    if not json_files:
        print(f"Skipping {dir} as it does not contain any JSON files.")
        
        continue
    select = sorted(json_files, key=lambda x: x.lower())[0].strip('.json')
    
    has_json = any(f.endswith('.json') for f in os.listdir(dir))
    if not has_json:
        print(f"Skipping {dir} as it does not contain any JSON files.")
        os.system(f'mv {dir} empty')
        continue
    
    # if content is empty, then delete folder. 
    
    try:
        keys = os.popen(f"jq -r 'recurse(.[]? // empty) | objects | keys_unsorted[]' {dir}*.json | sort | uniq -c | sort -nr").read()
        keynames = [i.split(' ')[-1] for i in keys.split('\n') if i.strip() != '']
        
        keynumbers = [int(i.split(' ')[-2]) for i in keys.split('\n') if i.strip() != '']
        avg = int(np.median(keynumbers))
        # print(keynames,keynumbers)
        
        
        
        different = [keynames[i] for i, x in enumerate(keynumbers) if x != avg]
        
        if different: print('The following keys are not present in all files:',different)


    except ValueError as e:
        # print(f"Error processing {dir}: {e}")
        continue

    
    dname = dir.strip('/').replace('-','_')
    pydantic = False
    
    match True:
        case _ if dname in DATA_DESCRIPTOR_CLASS_MAPPING:
            pydantic = dname  
        case _ if dname + '_new' in DATA_DESCRIPTOR_CLASS_MAPPING:
            pydantic = dname + '_new' 
        case _ if dir.strip('/') in DATA_DESCRIPTOR_CLASS_MAPPING:
            pydantic = dir.strip('/')
        case _:
            missing_pydantic.append(dname)  # none matched
            
            print(f"------ \n Adding {dname} to DATA_DESCRIPTOR_CLASS_MAPPING")
    
    
    
    if pydantic:
        bullets = bullet_pydantic(DATA_DESCRIPTOR_CLASS_MAPPING[pydantic])
    else:
        bullets = bullet_names(keynames)
    

    ldurl= ''
    content = get_path_url(dir).replace('wolfiex','wcrp-cmip')
    repo = get_repo_url().replace('wolfiex','wcrp-cmip')
    relpath = get_relative_path(dir)
    
    
    
    io = relpath.replace('src-data/',url2io(repo,'main',relpath))
    short = prefix_url(io)
    
    
    # print(content,repo)
    # print(relpath,short,io),
    # break
    
    
    info = f'''

<section id="info">


| Item | Reference |
| --- | --- |
| Type | `wrcp:{name}` |
| Pydantic class | [`{pydantic}`](https://github.com/ESGF/esgf-vocab/blob/main/src/esgvoc/api/data_descriptors/{pydantic}.py): {DATA_DESCRIPTOR_CLASS_MAPPING[pydantic].__name__ if pydantic else ' Not yet implemented'} |
| | |
| JSON-LD | `{short}` |
| Content | [{io}]({io}) |
| Developer Repo | [![Open in GitHub](https://img.shields.io/badge/Open-GitHub-blue?logo=github&style=flat-square)]({content}) |


</section>
    '''
    
    # Try to extract existing description from README file
    existing_description = ""
    readme_path = f"{dir}README.md"
    if os.path.exists(readme_path):
        with open(readme_path, 'r') as f:
            existing_content = f.read()
        existing_description = extract_description(existing_content)
    
    description = f'''

<section id="description">

# {name.title().replace('-',' ').replace(':',' : ')}  (universal)

## Description
{existing_description or ""}


</section>

'''    
    
    
    
    schema = f'''
<section id="schema">

## Content Schema

{bullets}




</section>   
'''
    

    
    usage = '''
<section id="usage">

## Usage

### Online Viewer 
To view a file in a browser use the content link with `.json` appended. 
eg. %(content)s/%(select)s.json

### Getting a File. 

A short example of how to integrate the computed ld file into your code. 

```python

import cmipld
cmipld.get( "%(short)s/%(select)s")

```

### Framing
Framing is a way we can filter the downloaded data to match what we want. 
```js
frame = {
            "@context": "%(url)s/_context_",
            "@type": "wcrp:%(dname)s",
            "keys we want": "",
            "@explicit": True

        }
        
```

```python

import cmipld
cmipld.frame( "%(short)s/%(select)s" , frame)

```
</section>

    '''%{'url': io, 'dname': name, 'short': short, 'content': content,'select': select}


    
    
    readme = f'''{description}{info}{schema}{usage}'''
    
    with open(f'{dir}README.md', 'w') as f:
        f.write(readme)
    

native-horizontal-grid-type/
------ 
 Adding native_horizontal_grid_type to DATA_DESCRIPTOR_CLASS_MAPPING
- **`validation-key`**
- **`ui-label`**
- **`type`**
- **`id`**
- **`description`**
- **`@context`**
make test for url2io
source-type/
make test for url2io
native-horizontal-grid-region/
------ 
 Adding native_horizontal_grid_region to DATA_DESCRIPTOR_CLASS_MAPPING
- **`validation-key`**
- **`ui-label`**
- **`type`**
- **`id`**
- **`description`**
- **`@context`**
make test for url2io
empty/
Skipping empty/ as it does not contain any JSON files.
model-calendar/
------ 
 Adding model_calendar to DATA_DESCRIPTOR_CLASS_MAPPING
- **`validation-key`**
- **`ui-label`**
- **`type`**
- **`id`**
- **`description`**
- **`@context`**
make test for url2io
license/
make test for url2io
frequency/
make test for url2io
activity/
The following keys are not present in all files: ['ui-label']
make test for url2io
native-horizontal-grid-temporal-refinement/
------ 
 Adding native_horizontal_grid_temp

In [8]:
print(usage)


<section id="usage">

## Usage

### Online Viewer 
To view a file in a browser use the content link with `.json` appended. 
eg. https://github.com/WCRP-CMIP/WCRP-universe/tree/main/src-data/model-family/acce.json

### Getting a File. 

A short example of how to integrate the computed ld file into your code. 

```python

import cmipld
cmipld.get( "universal:model-family/acce")

```

### Framing
Framing is a way we can filter the downloaded data to match what we want. 
```js
frame = {
            "@context": "https://wcrp-cmip.github.io/WCRP-universe/model-family/_context_",
            "@type": "wcrp:model-family",
            "keys we want": "",
            "@explicit": True

        }
        
```

```python

import cmipld
cmipld.frame( "universal:model-family/acce" , frame)

```
</section>

    


In [9]:
json_files = [f for f in os.listdir(dir) if f.endswith('.json')]
select = sorted(json_files, key=lambda x: x.lower())[0]
print(select)

access.json


In [10]:
frame = {
            "@context": "https://wcrp-cmip.github.io/WCRP-universe/model-family/_context_",
            "@type": "wcrp:model-family/",
            "keys we want": "",
            "@explicit": True

        }
        

In [11]:
import cmipld
vars(cmipld)


{'__name__': 'cmipld',
 '__doc__': None,
 '__package__': 'cmipld',
 '__loader__': <_frozen_importlib_external.SourceFileLoader at 0x113a08eb0>,
 '__spec__': ModuleSpec(name='cmipld', loader=<_frozen_importlib_external.SourceFileLoader object at 0x113a08eb0>, origin='/Users/daniel.ellis/WIPwork/CMIP-LD/cmipld/__init__.py', submodule_search_locations=['/Users/daniel.ellis/WIPwork/CMIP-LD/cmipld']),
 '__path__': ['/Users/daniel.ellis/WIPwork/CMIP-LD/cmipld'],
 '__file__': '/Users/daniel.ellis/WIPwork/CMIP-LD/cmipld/__init__.py',
 '__cached__': '/Users/daniel.ellis/WIPwork/CMIP-LD/cmipld/__pycache__/__init__.cpython-310.pyc',
 '__builtins__': {'__name__': 'builtins',
  '__doc__': "Built-in functions, exceptions, and other objects.\n\nNoteworthy: None is the `nil' object; Ellipsis represents `...' in slices.",
  '__package__': '',
  '__loader__': _frozen_importlib.BuiltinImporter,
  '__spec__': ModuleSpec(name='builtins', loader=<class '_frozen_importlib.BuiltinImporter'>, origin='built-in'

In [12]:

import cmipld
select = 'access.json'
a = f'universal:model-family/{select}'
print(a)
# cmipld.loader.set_cache_loader()

universal:model-family/access.json


In [13]:


# cmipld.loader.clear_cache()


cmipld.get(a,depth=3)


{'@context': 'universal:model-family/access.json',
 'id': 'universal:model-family/access',
 'type': 'wcrp:model-family',
 'description': 'Australian Community Climate and Earth System Simulator',
 'origin': 'universal:organisation/BoM/CSIRO (Bureau of Meteorology/Commonwealth Scientific and Industrial Research Organisation)',
 'validation-key': 'ACCESS'}

In [14]:

import pyld
from pyld import jsonld
jsonld.expand(a)

[{'https://wcrp-cmip.github.io/WCRP-universe/model-family/description': [{'@value': 'Australian Community Climate and Earth System Simulator'}],
  '@id': 'https://wcrp-cmip.github.io/WCRP-universe/model-family/access',
  'https://wcrp-cmip.github.io/WCRP-universe/model-family/origin': [{'@id': 'https://wcrp-cmip.github.io/WCRP-universe/organisation/BoM/CSIRO (Bureau of Meteorology/Commonwealth Scientific and Industrial Research Organisation)'}],
  '@type': ['wcrp:model-family'],
  'https://wcrp-cmip.github.io/WCRP-universe/model-family/validation-key': [{'@value': 'ACCESS'}]}]

In [15]:
import requests
ifile = requests.get(a).json()

# jsonld.expand(ifile)
ifile

{'description': 'Australian Community Climate and Earth System Simulator',
 'id': 'access',
 'origin': 'BoM/CSIRO (Bureau of Meteorology/Commonwealth Scientific and Industrial Research Organisation)',
 'type': 'wcrp:model-family',
 'validation-key': 'ACCESS',
 '@context': '_context_'}

In [16]:
cmipld.redir.list_redirects()

╭─────────────────────────────────────────────── Prefix Resolvers ────────────────────────────────────────────────╮
│ cf: https://wcrp-cmip.github.io/CF/                                                                             │
│ cmip7: https://wcrp-cmip.github.io/CMIP7-CVs/                                                                   │
│ obs4mips: https://wolfiex.github.io/obs4MIPs-cmor-tables-ld/                                                    │
│ universal: https://wcrp-cmip.github.io/WCRP-universe/                                                           │
│ variables: https://wcrp-cmip.github.io/MIP-variables/                                                           │
│ cmip6plus: https://wcrp-cmip.github.io/CMIP6Plus_CVs/                                                           │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

                                        Regex-Based URL Redirects (by Host)                                        
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                        Host                        ┃ Rules                                                      ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
└────────────────────────────────────────────────────┴────────────────────────────────────────────────────────────┘

In [17]:
cmipld.redir.test_redirect(a)

Original URL: universal:model-family/access.json
Final URL: https://wcrp-cmip.github.io/WCRP-universe/model-family/access.json
Status Code: 200
{
    "description": "Australian Community Climate and Earth System Simulator",
    "id": "access",
    "origin": "BoM/CSIRO (Bureau of Meteorology/Commonwealth Scientific and Industrial Research Organisation)",
    "type": "wcrp:model-family",
    "validation-key": "ACCESS",
    "@context": "_conte


In [18]:
# import tqdm,glob,json
# for f in tqdm.tqdm(glob.glob('*/*.json')):
#     data = json.load(open(f))
#     if '@context' not in data:
#         print(f"Adding context to {f}")
#         data['@context'] = f"_context_"
#         with open(f, 'w') as outfile:
#             json.dump(data, outfile, indent=4)

In [19]:
import cmipld
cmipld.get( "universal:frequency/1hr")

{'@context': 'universal:frequency/1hr',
 'id': 'universal:frequency/1hr',
 'type': ['wcrp:frequency', 'universal'],
 'description': 'sampled hourly',
 'long_name': '1 Hourly Frequency',
 'unit': 'hour',
 'validation-key': '1hr'}